## 通过akshare获取当日股票数据 并 存入pickle文件

In [1]:
import akshare as ak
from datetime import datetime, timedelta
from pathlib import Path
import pandas as pd
from pandas import DataFrame, Series
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import random
import pickle

import yfinance as yf

import common
import calculate

In [2]:
def get_today_stock_data():
    """
    获取每日股票date open high low close 数据
    可在 限制股票数量 进行测试
    """
    file_name = "./txt_lib/stock_code.txt"
    with open(file_name, "r") as file:
        stock_list = [line.strip() for line in file if line.strip()]

    # # 限制股票数量
    # stock_list = stock_list[:150]

    def get_stock_data(stock_code: str):
        """Fetch daily stock data for a stock code"""

        df = common.Download.download_with_retry(stock_code)

        # Convert date columns to strings
        for col in df.columns:
            if pd.api.types.is_datetime64_any_dtype(df[col]):
                df[col] = df[col].astype(str)
        return stock_code, df.to_dict()

    with ThreadPoolExecutor(max_workers=5) as executor:
        results = list(executor.map(get_stock_data, stock_list))

    # Convert results to a dictionary with stock codes as keys and DataFrame dictionaries as values
    data = {stock_code: df for stock_code, df in results}

    with open("./txt_lib/daily_df.pkl", "wb") as f:
        pickle.dump(data, f)

    print("Stock data has been saved to './txt_lib/daily_df.pkl'.")

In [3]:
get_today_stock_data()

Stock data has been saved to './txt_lib/daily_df.pkl'.


## TEST

In [4]:
from importlib import reload

reload(calculate)
reload(common)

<module 'common' from 'e:\\GRADUATE\\project\\stock_info\\common.py'>

In [5]:
stock_code = "sh600018"
df = common.Read_pickle.read_pickle_data(stock_code)

In [6]:
df.tail(7)

,date,open,high,low,close,volume,amount,outstanding_share,turnover
1806,2025-06-17,5.78,5.81,5.75,5.80,22424500.0,129640143.0,2.321458e+10,0.000966
1807,2025-06-18,5.80,5.84,5.77,5.82,23420700.0,136063941.0,2.321458e+10,0.001009
1808,2025-06-19,5.81,5.82,5.74,5.76,23247006.0,134059699.0,2.321458e+10,0.001001
1809,2025-06-20,5.76,5.87,5.75,5.84,37058825.0,216331778.0,2.321458e+10,0.001596
1810,2025-06-23,5.83,5.92,5.82,5.90,36945730.0,217272146.0,2.321458e+10,0.001591
1811,2025-06-24,5.90,5.91,5.81,5.84,38963311.0,227637695.0,2.321458e+10,0.001678
1812,2025-06-25,5.85,5.87,5.80,5.86,32994898.0,192546824.0,2.321458e+10,0.001421


In [7]:
5000 * 9 /60 /15

50.0